In [545]:
import scipy.io as sio
import scipy.stats as stats
import numpy as np
from numpy.linalg import *
import pandas as pd

In [546]:
def Single_Gaussian(x, mu, var):
    x_mu = x-mu
    b = x_mu**2 / var
    N = np.exp(-b/2)/ np.sqrt(var*2*np.pi)
    return N

In [547]:
def E_step(data , m , k , d , model):
    a = np.zeros((m,k,d))
    b = np.zeros((m,k,d))
    for j in range(k):
        a[:,j,:] = model['rho'] * Single_Gaussian(data,model['theta_mu'][j,:],model['theta_var'][j,:])
        b[:,j,:] = (1-model['rho']) * Single_Gaussian(data , model['lam_mu'] , model['lam_var'])
    c = a+b
    
    yu = c.prod(2)*model['alpha']
    w = yu / yu.sum(1).reshape(m,1)
    c[c==0] = 1
    w[np.isnan(w)] = 1

    u = np.zeros((m,k,d))
    v = np.zeros((m,k,d))
    for j in range(k):
        u[:,j,:] = (a[:,j,:]/c[:,j,:]) * w[:,j].reshape(m,1)
        v[:,j,:] = -u[:,j,:] + w[:,j].reshape(m,1)
    return w,u,v

In [548]:
def M_step(data, w, u, v, m, k, d ):
    model = {}
    #alpha = w.sum(0)/m
    yu = np.maximum(w.sum(0)-d,0)
    alpha = yu / yu.sum()

    #rho = u.sum(1).sum(0)/m
    rho = (np.maximum(u.sum(1).sum(0)-k,0)) / (np.maximum(u.sum(1).sum(0)-k,0) + np.maximum(v.sum(1).sum(0)-1,0))

    theta_mu = np.zeros((k,d))
    theta_var = np.zeros((k,d))
    lam_mu = np.zeros(d)
    lam_var = np.zeros(d)
    
    for l in range(d):
        for j in range(k):
            if (u[:,j,l].sum()==0):
                alpha[j]=0
            theta_mu[j,l] = np.dot(u[:,j,l].T , data[:,l])/u[:,j,l].sum()
            theta_var[j,l] = np.dot(u[:,j,l].T,((data[:,l]-theta_mu[j,l])**2))/u[:,j,l].sum()
    theta_var[theta_var==0] = 0.0001
    
    
    for l in range(d):
        e = 0
        for i in range(m):
            e = e + v[i,:,l].sum() * data[i,l]
        f = v[:,:,l]
        lam_mu[l] = e/ f.sum()
        if(f.sum()==0):
            lam_mu[l] = (e+0.0001)/(f.sum()+0.0001)
            rho[l] = rho[l]*0.9
    
    for l in range(d):
        e = 0
        for i in range(m):
            e = e + v[i,:,l].sum() * ((data[i,l]-lam_mu[l])**2)
        f = v[:,:,l]
        lam_var[l] = e/ f.sum()
        if(f.sum()==0 or e==0):
            lam_var[l] = (e+0.0001)/(f.sum()+0.0001)
            rho[l] = rho[l]*0.9
    
    #theta_mu = (u*data.reshape(m,1,d)).sum(0)/u.sum(0)
    #theta_var = np.zeros((k,d))
    #for j in range(k):
    #    theta_var[j,:] = (u[:,j,:] * ((data-theta_mu[j:j+1,:])**2)).sum(0)/u[:,j,:].sum(0)
    #    
    #theta_var[theta_var==0] = 0.0001
    #yu = v.sum(1).sum(0)
    #rho[yu==0] = rho[yu==0]*0.9
    #yu[yu==0] = 0.001
    #lam_mu = (v.sum(1)*data).sum(0)/yu
    #lam_var = (v.sum(1) * ((data-lam_mu)**2)).sum(0)/yu
    
    model['rho'] = rho
    model['alpha'] = alpha
    model['theta_mu'] = theta_mu
    model['theta_var'] = theta_var
    model['lam_mu'] = lam_mu
    model['lam_var'] = lam_var
    
    return model

In [549]:
def move_superfluous(model, k, d , data):
    #move the superfluous feature and component
    rho = model['rho']
    alpha = model['alpha']
    theta_mu = model['theta_mu']
    theta_var = model['theta_var']
    lam_mu = model['lam_mu']
    lam_var = model['lam_var']
    
    alpha0 = alpha<0.001
    if alpha0.sum()!=0:
        theta_mu = theta_mu[~alpha0,:]
        theta_var = theta_var[~alpha0,:]
        alpha = alpha[~alpha0]
        k = k-alpha0.sum()

    rho1 = rho==1
    if rho1.sum()!=0:
        lam_mu[rho1] = lam_mu[rho1]*0.9
        lam_var[rho1] = lam_var[rho1]*0.9
        rho[rho1] = 0.9

    rho0 = rho<0.001
    if rho0.sum()!=0:
        lam_mu = lam_mu[~rho0]
        lam_var = lam_var[~rho0]
        theta_mu = theta_mu[:,~rho0]
        theta_var = theta_var[:,~rho0]
        data = data[:,~rho0]
        rho = rho[~rho0]
        d = d-(rho0).sum()
    
    model['rho'] = rho
    model['alpha'] = alpha
    model['theta_mu'] = theta_mu
    model['theta_var'] = theta_var
    model['lam_mu'] = lam_mu
    model['lam_var'] = lam_var
    return model,k,d,data

In [550]:
def MML(data, m, d, k, model):
    # compute the cost
    a = np.zeros((m,k,d))
    b = np.zeros((m,k,d))
    for j in range(k):
        a[:,j,:] = model['rho'] * Single_Gaussian(data,model['theta_mu'][j,:],model['theta_var'][j,:])
        b[:,j,:] = (1-model['rho']) * Single_Gaussian(data , model['lam_mu'] , model['lam_var'])
    c = a+b
    c[c>1] = 1
    yu = c.prod(2)*model['alpha']
    yu = yu.sum()
    yu = np.log(yu)
    #cost = -yu/m
    cost = -yu + d*np.log(alpha).sum() + np.log(1-rho).sum() + k*np.log(rho).sum()
    return cost

In [551]:
np.random.seed(15)
key = np.array([[0.0,1.0,6.0,7.0],[3.0,9.0,4.0,10.0]])
label = 4
key = np.row_stack( (key,np.random.normal(size=(8,4))) )
print(key)
std = np.array([1,1,1,1])
data =  key[:,0:1] + np.random.normal(size=(10,200))*std[0]
for i in range(1,label):
    data = np.column_stack( (data , key[:,i:i+1] + np.random.normal(size=(10,200))*std[i] ) )
data = data.T

[[  0.           1.           6.           7.        ]
 [  3.           9.           4.          10.        ]
 [ -0.31232848   0.33928471  -0.15590853  -0.50178967]
 [  0.23556889  -1.76360526  -1.09586204  -1.08776574]
 [ -0.30517005  -0.47374837  -0.20059454   0.35519677]
 [  0.68951772   0.41058968  -0.56497844   0.59939069]
 [ -0.16293631   1.6002145    0.6816272    0.0148801 ]
 [ -0.08777963  -0.98211784   0.12169048  -1.13743729]
 [  0.34900258  -1.85851316  -1.16718189   1.42489683]
 [  1.49656536   1.28993206  -1.81174527  -1.49830721]]


In [552]:
[m,d] = data.shape
k = 100
kmin = 4
cost =  0
oldcost = 10000

model = {}
#theta_mu = np.random.normal(size=(k,d))
kk = np.arange(m)
np.random.shuffle(kk)
model['theta_mu'] = data[kk[:k],:]
model['theta_var'] = np.random.uniform(size=(k,d))
model['lam_mu'] = data.mean(0)
model['lam_var'] = np.random.uniform(size=(d,1)).reshape(d)
model['rho'] = np.ones((1,d))*0.5
model['alpha'] = np.random.uniform(size=(1,k))
model['alpha'] = model['alpha']/model['alpha'].sum()

In [553]:
model['alpha']

array([[  1.32293382e-02,   1.57302193e-02,   1.29829359e-02,
          7.46900940e-03,   1.21090666e-02,   1.18229176e-02,
          3.60281188e-03,   1.41005905e-02,   1.50132651e-02,
          4.29517078e-03,   1.93570632e-02,   9.27868559e-03,
          1.28042258e-02,   1.36269496e-02,   1.71201415e-02,
          1.93832436e-02,   1.14521692e-02,   1.45683003e-02,
          1.07093719e-03,   7.37117536e-04,   1.94716846e-02,
          1.59483116e-03,   1.59149143e-02,   1.13155499e-02,
          1.89235611e-02,   1.65456874e-02,   1.15897516e-02,
          1.73443063e-03,   1.41604559e-02,   6.47767022e-03,
          4.51776637e-05,   3.12432214e-04,   7.03616052e-03,
          1.57938522e-02,   1.86220235e-02,   4.17944627e-03,
          4.81234341e-03,   6.17463216e-03,   8.67225359e-04,
          3.16596023e-03,   7.84788288e-04,   4.36818794e-03,
          1.14161082e-02,   1.51134014e-02,   1.73596436e-02,
          1.71443554e-02,   8.76785431e-03,   6.50454506e-03,
        

In [554]:
savemodel = []
while(k>kmin):
    print(k)
    step = 0
    oldcost = 10000
    while( (abs(cost-oldcost)>0.001) and (step<100) ):
        step = step+1
        [ w, u, v ] = E_step(data , m , k , d , model)
        model = M_step(data, w, u, v, m, k, d)

        [model,k,d,data] = move_superfluous(model, k, d,data)
        savemodel.append(model)
        print(k)
        if(k<kmin):
            model = savemodel[-2]
            k = model['alpha'].shape[0]
            break
        oldcost = cost
        cost = MML(data, m, d, k, model)
    
    if( (step==100) and (abs(cost-oldcost)>0.001) and (k>kmin)):
        alpha0 = model['alpha'].argmin()
        model['theta_mu'] = np.delete(model['theta_mu'], alpha0 , axis=0)
        model['theta_var'] = np.delete(model['theta_var'], alpha0 , axis=0)
        model['alpha'] = np.delete(model['alpha'], alpha0 , axis=0)
        savemodel.append(model)
        k = k-1
        
        

100
27
19
18
15
14
14
12
11
11
10
10
10
10
9
9
8
8
8
7
7
7
7
7
7
7
7
7
7
7
7
7
7
6
6
6
6
6
6
6
6
6
6
6
6
5
5
4
4
4
4
4
4
4


In [555]:
model['theta_mu']

array([[  6.95561148,  10.06410052,  -0.89899912,  -1.16984461,
          1.27598255,   1.00800947,  -0.29394819,  -1.91123452,
          1.53421212,  -1.61575547],
       [  5.97391485,   3.94663403,  -0.05515349,  -0.86910748,
         -1.0799806 ,  -1.06697468,   0.80827338,   0.49547804,
         -1.19710674,  -2.02364428],
       [  0.97192785,   9.04253114,   1.19355573,  -2.16332948,
         -1.37968251,   0.38889422,   1.95428889,  -1.7296835 ,
         -1.90618183,   1.31233134],
       [ -0.01069196,   2.95255168,  -0.86325302,   0.52070422,
         -1.3011888 ,   1.22817718,  -0.49420839,  -0.15282419,
          0.31796144,   1.61759984]])

In [556]:
key

array([[  0.        ,   1.        ,   6.        ,   7.        ],
       [  3.        ,   9.        ,   4.        ,  10.        ],
       [ -0.31232848,   0.33928471,  -0.15590853,  -0.50178967],
       [  0.23556889,  -1.76360526,  -1.09586204,  -1.08776574],
       [ -0.30517005,  -0.47374837,  -0.20059454,   0.35519677],
       [  0.68951772,   0.41058968,  -0.56497844,   0.59939069],
       [ -0.16293631,   1.6002145 ,   0.6816272 ,   0.0148801 ],
       [ -0.08777963,  -0.98211784,   0.12169048,  -1.13743729],
       [  0.34900258,  -1.85851316,  -1.16718189,   1.42489683],
       [  1.49656536,   1.28993206,  -1.81174527,  -1.49830721]])

In [53]:
a = np.zeros((m,k,d))
b = np.zeros((m,k,d))
for j in range(k):
    for l in range(d):
        a[:,j,l] = rho[l] * stats.norm.pdf(data[:,l],theta_mu[j,l],theta_var[j,l])
        b[:,j,l] = (1-rho[l]) * stats.norm.pdf(data[:,l] , lam_mu[l] , lam_var[l])
c = a+b
c[c>0] = 1
yu = c.prod(2)*alpha
yu = yu.sum(1).sum(0)
yu = np.log(yu)
#cost = -yu/m
cost = -yu + d*np.log(alpha).sum() + np.log(1-rho).sum() + k*np.log(rho).sum()

In [54]:
np.log(alpha).sum()

-102.26131709656298

In [48]:
np.maximum([1,9],[3,6])

array([3, 9])

In [360]:
a = np.array([[[1,2],[3,4],[4,8]],[[5,6],[7,8],[8,7]]])
a.sum()

63

In [135]:
theta_var = np.random.uniform(size=(k,d))
a = np.random.shuffle(np.arange(10))
print(a)

None


In [134]:
arr = np.arange(10)
np.random.shuffle(np.arange(10))
arr

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [137]:
theta_mu.shape

(100, 10)